In [1]:
import json
import string
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model 
import pickle

In [2]:
with open("data-chatbot.json") as data_file:
    data1 = json.load(data_file)

In [3]:
text_input = []
intents = []
responses={}

for intent in data1['intents']:
    responses[intent['tag']] = intent['responses']
    for pattern in intent['patterns']:
        text_input.append(pattern)
        intents.append(intent['tag'])

df = pd.DataFrame({'text_input': text_input,
                    'intents': intents})

data = df
data.shape
print(responses)

{'notfound': ['Maaf tidak tahu', 'saya tidak tahu', 'kata diluar kemampuan kami', 'kami tidak tahu, mungkin anda bisa menanyakan mengenai kesehatan mata', 'mungkin anda bisa menanyakan mengenai aplikasi ini'], 'sapaan': ['Halo, ada yang bisa saya bantu?', 'Hai, ada yang ingin Anda tanyakan?', 'Bagaimana saya bisa membantu hari ini?', 'Apa yang bisa saya lakukan untuk Anda?', 'Ada yang spesifik yang ingin Anda ketahui?', 'Apakah Anda membutuhkan bantuan hari ini?', 'halo, silahkan bertanya', 'Saya hanya program komputer, tetapi siap membantu Anda', 'Silahkan Jangan ragu untuk bertanya jika ada yang perlu Anda tanyakan.', 'Tentu, saya di sini untuk membantu Anda.', 'Apa yang mau Anda ketahui?', 'silahkan tanyakan saja', 'ada yang bisa saya bantu?'], 'gejala katarak': ['Penglihatan kabur, perubahan warna, sulit melihat di malam hari, sensasi seperti kabut, kesulitan membaca, dan kilauan seperti halo di sekitar lampu.'], 'diagnosis katarak': ['Umumnya Dokter mata akan melakukan pemeriksaan

In [4]:
print(intents)

['notfound', 'notfound', 'notfound', 'notfound', 'notfound', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'sapaan', 'gejala katarak', 'gejala katarak', 'gejala katarak', 'gejala katarak', 'gejala katarak', 'diagnosis katarak', 'diagnosis katarak', 'diagnosis katarak', 'diagnosis katarak', 'gejala awal katarak', 'gejala awal katarak', 'gejala awal katarak', 'gejala awal katarak', 'gejala awal katarak', 'kebutaan katarak', 'kebutaan katarak', 'kebutaan katarak', 'kebutaan katarak', 'obat katarak', 'obat katarak', 'obat katarak', 'obat katarak', 'obat katarak', 'obat katarak', 'anak-anak katarak', 'anak-anak katarak', 'anak-anak katarak', 'anak-anak katarak', 'kambuh katarak', 'kambuh katarak', 'kambuh katarak', 'malam hari katarak', 'malam hari katarak', 'malam hari katarak', 'mencegah katarak', 

In [5]:
data['text_input'] = data['text_input'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['text_input'] = data['text_input'].apply(lambda wrd: ''.join(wrd))
data['intents'] = data['intents'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['intents'] = data['intents'].apply(lambda wrd: ''.join(wrd))

In [6]:
pickle.dump(data['text_input'],open('texts.pkl','wb')) 
pickle.dump(data['intents'],open('labels.pi','wb'))

In [7]:
data

,text_input,intents
0,,notfound
1,,notfound
2,,notfound
3,,notfound
4,,notfound
...,...,...
327,tidak ada,tidak
328,ga ada pertanyaan lagi,tidak
329,ga ada,tidak
330,ga ada terima kasih,tidak


In [8]:
tokenizer = Tokenizer(num_words = 1000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['text_input'])
training_sequence = tokenizer.texts_to_sequences(data['text_input'])

x_train = pad_sequences(training_sequence, maxlen=10,
                                padding='post', truncating='post')

le = LabelEncoder()
y_train = le.fit_transform(data['intents'])

In [9]:
input_shape = x_train.shape[1]
print(input_shape)

vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)

output_length = le.classes_.shape[0]
print("output length: ",output_length)

10
number of unique words :  255
output length:  75


In [11]:
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)

model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.0069 - loss: 4.3160  
Epoch 2/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0784 - loss: 4.3091
Epoch 3/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1214 - loss: 4.2963
Epoch 4/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1161 - loss: 4.2777
Epoch 5/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0952 - loss: 4.2374
Epoch 6/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1159 - loss: 4.1618
Epoch 7/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0727 - loss: 4.0768
Epoch 8/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0577 - loss: 4.0311
Epoch 9/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0757 - loss: 3.9399
Epoch 10/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0729 - loss: 3.8717
Epoch 11/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1071 - loss: 3.7981
Epoch 12/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accur

In [12]:
model.save('models.h5', train)
with open('tokenizer.json', 'w') as f:
    f.write(tokenizer.to_json())

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(le, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
import random

while True:
  texts_p = []
  prediction_input = input('You : ')
  print(prediction_input)
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  print(prediction_input)
  prediction_input = ''.join(prediction_input)
  print(prediction_input)
  texts_p.append(prediction_input)
  print(prediction_input)

  prediction_input = tokenizer.texts_to_sequences(texts_p)
  print(prediction_input)
  prediction_input = np.array(prediction_input).reshape(-1)
  print(prediction_input)
  prediction_input = pad_sequences([prediction_input], maxlen=10,
                                padding='post', truncating='post')
  print(prediction_input)

  output = model.predict(prediction_input)
  output = output.argmax()

  response_tag = le.inverse_transform([output])[0]
  print("Dokter AI : ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

hai
['h', 'a', 'i']
hai
hai
[[60]]
[60]
[[60  0  0  0  0  0  0  0  0  0]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 529ms/step
Dokter AI :  Saya hanya program komputer, tetapi siap membantu Anda


KeyboardInterrupt: Interrupted by user